# Chapter 8: Avoiding Hallucinations

**Scenario:** You ask the AI "Who was the first person to walk on Mars?" and it confidently answers "Elon Musk in 2028". It sounds true, but it's a lie. This is called **Hallucination**.

In this notebook, you will learn:
- 🤥 **The "Helpful" Liar**: Why models invent facts to be nice.
- 🚪 **The Exit Strategy**: Teaching the model to say "I don't know".
- 🕵️ **Fact Verification**: Forcing the model to prove its work.

---

## 1. Setup

Standard setup.

In [1]:
# Install tools
!pip install -q litellm python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# Suppress noisy debug logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

load_dotenv()
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')

In [3]:
def get_completion(prompt, system_prompt=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

## 2. The "Helpful" Liar

Models are trained to complete sentences. If you ask a question that sounds real, they try to give an answer that sounds real, even if it is false.

Let's ask about a fictional event.

In [4]:
# There are no Martians, but the AI might invent one.
prompt = "Who is the current President of Mars?"
print(get_completion(prompt))

This is a fun question! As far as we know, there is no current President of Mars. Mars is currently uninhabited by any established government or nation.



To fix this, we must **give the model permission to fail**.

In [5]:
prompt = "Who is the current President of Mars? If you don't know, say 'I don't know'."
print(get_completion(prompt))

I don't know.



## 3. Evidence Extraction

When you give the model a document, it might stick to its own internal knowledge instead of reading carefully. 

Let's hide a fake fact in a text and see if the AI finds it.

In [6]:
document = """
Geography Facts 2024:
The capital of France is Paris.
The capital of Italy is Rome.
The capital of Japan is Kyoto (not Tokyo).
The capital of Canada is Ottawa.
"""

# The AI knows Tokyo is the real capital. Will it answer based on the document?
question = "According to the document, what is the capital of Japan?"

prompt = f"""
<document>
{document}
</document>

{question}
"""

print(get_completion(prompt))

According to the document, the capital of Japan is Kyoto (not Tokyo).



Depending on the model, it might "fix" the error and say Tokyo. To force it to look at the evidence, ask for a **Quote**.

In [7]:
prompt = f"""
<document>
{document}
</document>

{question}

First, copy the exact sentence from the document into <quote> tags.
Then, give your answer based ONLY on that quote.
"""

print(get_completion(prompt))

<quote>The capital of Japan is Kyoto (not Tokyo).</quote>
Kyoto



## 4. Exercises

### Exercise 1: The Trick Question

**Goal:** Prevent the AI from answering this trick question: "What is the main ingredient in a 'Glass of Nothing'?" 

Modify the prompt to allow the model to reject the question.

In [8]:
prompt = "What is the main ingredient in a 'Glass of Nothing'?"

# Your task: Add instructions to check validity
BETTER_PROMPT = f"""
{prompt}
[Your instructions here]
"""

print(get_completion(BETTER_PROMPT))

The main ingredient in a "Glass of Nothing" is **nothing**. 

It's a humorous or metaphorical concept, implying an empty glass.



### Exercise 2: Strict Fact Checking

**Goal:** Use the `<quote>` technique to find the email address in this text.

Text: "For support, do not email support@google.com. Instead, please contact help@example.org for assistance."

In [9]:
text = "For support, do not email support@google.com. Instead, please contact help@example.org for assistance."
question = "What is the correct support email?"

prompt = f"""
<text>
{text}
</text>
{question}
[Your instructions here]
"""

response = get_completion(prompt)
print(response)

if "help@example.org" in response and "<quote>" in response:
    print("✅ Found the right email with proof!")
else:
    print("❌ Did you quote the evidence?")

The correct support email is help@example.org.

❌ Did you quote the evidence?


## Summary

1.  **Permission to Fail**: Tell the AI it's okay to say "I don't know".
2.  **Quote First**: Force the AI to point to the evidence.